In [16]:
import json

from jp_prefecture.jp_cities import jp_cities as jp
from jre_utils.datapath import DATA_DIRECTORY_PATH

In [17]:
area_code_to_area_path = f"{DATA_DIRECTORY_PATH}/core_scraped/area_code_to_area.json"
with open(area_code_to_area_path) as fd:
     area_code_to_area = json.load(fd)
     print(f"E.g. Maps 1100 to {area_code_to_area["1100"]}") 

E.g. Maps 1100 to Hokkaido Sapporo-shi


In [18]:
# help(jp)

In [32]:
# Helpers

def get_geocode(area_code):
    # area_code -> (latitude, longitude)
    try:
        return tuple(jp.citycode2geodetic(area_code)) 
    except:
        print(f"Could not find geocode for {area_code}")
        return (0, 0)

def get_euclidian_distance(geocode1, geocode2):
    # lat1, lon1, lat2, lon2 -> distance
    lat1, lon1 = geocode1
    lat2, lon2 = geocode2
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2)**0.5

def find_neighbours(area_code, area_to_geocode, n):
    geocode = area_to_geocode[area_code]
    area_to_distances = { area : get_euclidian_distance(geocode, area_geocode) for area, area_geocode in area_to_geocode.items() if area != area_code }
    return sorted(area_to_distances, key=area_to_distances.get)[:n]

# get_euclidian_distance(geocode(13101), geocode(13102))

In [36]:
# Smoothing parameters

n_neighbors = 5

all_area_codes = list(area_code_to_area.keys())
area_to_geocode = {area_code: get_geocode(area_code) for area_code in all_area_codes}
area_to_neighbours = {area_code: find_neighbours(area_code, area_to_geocode, 5) for area_code in all_area_codes}

Could not find geocode for 13362
Could not find geocode for 43506


In [37]:
# Get Longitudes and Latitudes
# For Area, get 5 closest neighbors

area_to_neighbours

{'1100': ['1107', '1101', '1109', '1106', '1105'],
 '1101': ['1107', '1105', '1106', '1103', '1100'],
 '1102': ['1103', '1104', '1105', '1107', '1101'],
 '1103': ['1102', '1104', '1105', '1107', '1101'],
 '1104': ['1103', '1108', '1105', '1102', '1101'],
 '1105': ['1101', '1107', '1104', '1103', '1106'],
 '1106': ['1110', '1101', '1105', '1107', '1100'],
 '1107': ['1101', '1105', '1103', '1109', '1102'],
 '1108': ['1104', '1234', '1110', '1423', '1217'],
 '1109': ['1107', '1100', '1101', '1102', '1103'],
 '1110': ['1106', '1108', '1105', '1104', '1101'],
 '1202': ['1236', '1337', '1334', '1345', '1343'],
 '1203': ['1409', '1408', '1109', '1100', '1407'],
 '1204': ['1452', '1453', '1454', '1455', '1458'],
 '1205': ['1233', '1230', '1575', '1584', '1397'],
 '1206': ['1661', '1667', '1664', '1668', '1662'],
 '1207': ['1643', '1638', '1631', '1644', '1639'],
 '1208': ['1549', '1550', '1552', '1544', '1648'],
 '1209': ['1429', '1427', '1222', '1215', '1210'],
 '1210': ['1304', '1222', '1429